In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import learning.min_regret_strategy_pb2
import numpy as np

In [ ]:
strategy_file = '/tmp/pokerbot_checkpoint_7100000.pb'
with open(strategy_file, 'rb') as file_in:
    strategy = learning.min_regret_strategy_pb2.MinRegretStrategy()
    strategy.ParseFromString(file_in.read())
    

In [ ]:
def parse_infoset_id(id_num):
    out = {}
    action_int = (id_num >> 40)
    out['actions'] = []
    while action_int > 0:
        action_idx = action_int & 0x0F
        # we increment all indexes by one to be able to distiguish no action from the first action
        # however, there is also a gap. The action list is:
        # [Fold, Check, Call, Raise, RaisePot, AllIn], where Raise isn't an action that is considered
        action_from_action_idx = ['NULL', 'Fold', 'Check', 'Call', 'Raise', 'RaisePot', 'AllIn']
        out['actions'].append(action_from_action_idx[action_idx])
        action_int = action_int >> 4
    
    out['betting_round'] = (id_num >> 32) & 0xFF
    if out['betting_round'] == 0:
        out['ranks'] = ((id_num >> 16) & 0xFFFF)
        out['suits'] = (id_num & 0xFFFF)
    else:
        out['hand_strength_bin'] = ((id_num >> 8) & 0xFF)
        out['negative_potential_bin'] = ((id_num >> 4) & 0x0F)
        out['positive_potential_bin'] = (id_num & 0x0F)
    return out

In [ ]:
len(strategy.infoset_counts)

In [ ]:
strategy.infoset_counts[0]

In [ ]:
iter_counts = sorted([x.iter_count for x in strategy.infoset_counts])
frac = np.linspace(0, 1.0, len(iter_counts))

In [ ]:
plt.figure()
plt.plot(iter_counts, frac)
plt.ylabel('Fraction of bins')
plt.xlabel('Counts per bin')

In [ ]:
max_bin = max(strategy.infoset_counts, key=lambda x: x.iter_count)

In [ ]:
max_bin

In [ ]:
parse_infoset_id(max_bin.id_num)

In [ ]:
for infoset_counts in strategy.infoset_counts:
    parsed_id = parse_infoset_id(infoset_counts.id_num)
    if len(parsed_id['actions']) == 6:
        print('found one with full actions!')
        break

In [ ]:
strategy.actions

In [ ]:
infoset_counts

In [ ]:
parsed_id

## 